In [ ]:
import matplotlib.patches as patches
import matplotlib.pyplot as plt
import SimpleITK as sitk
import os
import numpy as np
import pandas as pd

In [ ]:
def myshow(img):
    nda = sitk.GetArrayFromImage(img)
    plt.imshow(nda, cmap='gray')
    plt.axis('off')

def myshow_with_rectangle(img, rect_color, rect_linewidth=10, ax=None):
    if ax is None:
        fig, ax = plt.subplots()

    nda = sitk.GetArrayFromImage(img)
    ax.imshow(nda, cmap='gray')
    ax.axis('off')

    # Create a Rectangle patch
    rect = patches.Rectangle((0, 0), img.GetWidth(), img.GetHeight(), linewidth=rect_linewidth, edgecolor=rect_color, facecolor='none', fill=False)

    # Add the patch to the Axes
    ax.add_patch(rect)

def display_sample_images(image_frames_info,
                          images_rootpath,
                          img_path_column,
                          highlight_column=None,
                          savefig_filepath=None,
                          ga_col = None,
                          plt_title = None):
    image_frames_info = image_frames_info.sample(n=min(image_frames_info.shape[0], 100), random_state = 42)
    image_frames_info = image_frames_info.sort_values(by=ga_col, ascending=False)
    n_points = image_frames_info.shape[0]
    nrows = int(np.ceil(np.sqrt(n_points)))
    ncols = int(np.ceil(n_points/nrows))
    figsize = (ncols*4, nrows*4)

    fig, ax = plt.subplots(figsize=figsize, nrows=nrows, ncols=ncols)
    for i, x in enumerate(image_frames_info.iterrows(), 1):   #.sort_index()
        x = x[1]
        img_path = os.path.join(images_rootpath, x[img_path_column])
        img_T1 = sitk.ReadImage(img_path)
        # print(highlight_column)
        if highlight_column is not None:
            # print("Entered if1")
            # print(x[highlight_column])
            # show image with a bounding box around it
            myshow_with_rectangle(img_T1, 
                                  'lightgreen' if x[highlight_column] else 'red',  # line color
                                  15, # line width
                                  ax=plt.subplot(nrows,ncols,i))
        else:
            # plot image without BB
            plt.subplot(nrows,ncols,i)
            myshow(img_T1)
        if x.image_title is not None:
            plt.title(f'{x.image_title}', fontsize=20, fontweight="bold", color='steelblue')
    
    fig.suptitle(plt_title, fontsize=54, color='Black')
    plt.tight_layout()
    fig.subplots_adjust(top=0.95)

    if savefig_filepath is not None:
        plt.savefig(savefig_filepath)
    else:
        # plt.title("Hello World!")
        plt.show()



In [ ]:
df = pd.read_csv("/mnt/raid/home/ayrisbud/us-famli-pl/src/predictedClassifier.csv")
classif_error_column = ['No structures visible', 'Head Visible',
       'Abdomen Visible', 'Amniotic fluid visible', 
       'Placenta visible', 'Fetus or CRL visible']  # set this to name of column containing boolean value, True if it's a classification error
print(df.columns)

# for i in range(len(classif_error_column)):
# #     print("Entered i: ")
#     img_path_column = 'img_path'
#     ga_column_name = 'ga_boe_weeks'
#     originalCol = classif_error_column[i]
#     predictedCol = classif_error_column[i] + "_predicted"
#     compareCol = classif_error_column[i] + "_compare"
# #     print("Entered i: ", originalCol, " ", predictedCol, " ", compareCol)
#     df[ga_column_name] = (df.ga_boe / 7.0).fillna(value=-1)
#     # df['is_measurable_head'] = df['Head Measurable'] == "YES"
#     df[compareCol] = [True if original == predicted else False for original, predicted in zip(df[originalCol], df[predictedCol])]
#     df['image_title'] = df.apply(lambda x: '{}weeks|{}'.format(np.round(x[ga_column_name],2), x.tag), axis=1)
#     df = df.sort_values(by=ga_column_name, ascending=True)
#     # print(df.shape)
#     # print(df.columns)
#     # print(df.head)

#     data_rootpath = "/mnt/raid/C1_ML_Analysis/"
#     display_sample_images(image_frames_info=df, 
#                                     images_rootpath=data_rootpath, 
#                                     img_path_column=img_path_column,
#                                     highlight_column = compareCol,
#                                     savefig_filepath=None
#                                     )

In [ ]:
img_path_column = 'img_path'
ga_column_name = 'ga_boe_weeks'
originalCol = classif_error_column[0]
predictedCol = classif_error_column[0] + "_predicted"
compareCol = classif_error_column[0] + "_compare"
# print("Entered i: ", originalCol, " ", predictedCol, " ", compareCol)
df[ga_column_name] = (df.ga_boe / 7.0).fillna(value=-1)
# df['is_measurable_head'] = df['Head Measurable'] == "YES"
df[compareCol] = [True if original == predicted else False for original, predicted in zip(df[originalCol], df[predictedCol])]
df['image_title'] = df.apply(lambda x: '{}weeks|{}'.format(np.round(x[ga_column_name],2), x.tag), axis=1)
# df = df.sort_values(by=ga_column_name, ascending=False)
# print(df[["ga_boe_weeks", "image_title"]].head())
# print(df.shape)
# print(df.columns)
# print(df.head)
condition1 = df[originalCol] == 1
condition2 = df[originalCol] == 0
df1 = df[condition1]
df2 = df[condition2]

data_rootpath = "/mnt/raid/C1_ML_Analysis/"
display_sample_images(image_frames_info=df1, 
                                images_rootpath=data_rootpath, 
                                img_path_column=img_path_column,
                                highlight_column = compareCol,
                                savefig_filepath=None,
                                ga_col = ga_column_name,
                                plt_title = originalCol + " Ground Truth = 1",
                                savefig_filepath = "/mnt/raid/home/ayrisbud/us-famli-pl/src/classification_samples/" + originalCol + "_Ground Truth_1.png"
                                )

display_sample_images(image_frames_info=df2, 
                                images_rootpath=data_rootpath, 
                                img_path_column=img_path_column,
                                highlight_column = compareCol,
                                savefig_filepath=None,
                                ga_col = ga_column_name,
                                plt_title = originalCol + " Ground Truth = 0",
                                savefig_filepath = "/mnt/raid/home/ayrisbud/us-famli-pl/src/classification_samples/" + originalCol + "_Ground Truth_0.png"
                                )

In [ ]:
img_path_column = 'img_path'
ga_column_name = 'ga_boe_weeks'
originalCol = classif_error_column[1]
predictedCol = classif_error_column[1] + "_predicted"
compareCol = classif_error_column[1] + "_compare"
# print("Entered i: ", originalCol, " ", predictedCol, " ", compareCol)
df[ga_column_name] = (df.ga_boe / 7.0).fillna(value=-1)
# df['is_measurable_head'] = df['Head Measurable'] == "YES"
df[compareCol] = [True if original == predicted else False for original, predicted in zip(df[originalCol], df[predictedCol])]
df['image_title'] = df.apply(lambda x: '{}weeks|{}'.format(np.round(x[ga_column_name],2), x.tag), axis=1)
# df = df.sort_values(by=ga_column_name, ascending=False)
# print(df[["ga_boe_weeks", "image_title"]].head())
# print(df.shape)
# print(df.columns)
# print(df.head)
condition1 = df[originalCol] == 1
condition2 = df[originalCol] == 0
df1 = df[condition1]
df2 = df[condition2]

data_rootpath = "/mnt/raid/C1_ML_Analysis/"
display_sample_images(image_frames_info=df1, 
                                images_rootpath=data_rootpath, 
                                img_path_column=img_path_column,
                                highlight_column = compareCol,
                                savefig_filepath=None,
                                ga_col = ga_column_name,
                                plt_title = originalCol + " Ground Truth = 1",
                                savefig_filepath = "/mnt/raid/home/ayrisbud/us-famli-pl/src/classification_samples/" + originalCol + "_Ground Truth_1.png"
                                )

display_sample_images(image_frames_info=df2, 
                                images_rootpath=data_rootpath, 
                                img_path_column=img_path_column,
                                highlight_column = compareCol,
                                savefig_filepath=None,
                                ga_col = ga_column_name,
                                plt_title = originalCol + " Ground Truth = 0",
                                savefig_filepath = "/mnt/raid/home/ayrisbud/us-famli-pl/src/classification_samples/" + originalCol + "_Ground Truth_0.png"
                                )

In [ ]:
img_path_column = 'img_path'
ga_column_name = 'ga_boe_weeks'
originalCol = classif_error_column[2]
predictedCol = classif_error_column[2] + "_predicted"
compareCol = classif_error_column[2] + "_compare"
# print("Entered i: ", originalCol, " ", predictedCol, " ", compareCol)
df[ga_column_name] = (df.ga_boe / 7.0).fillna(value=-1)
# df['is_measurable_head'] = df['Head Measurable'] == "YES"
df[compareCol] = [True if original == predicted else False for original, predicted in zip(df[originalCol], df[predictedCol])]
df['image_title'] = df.apply(lambda x: '{}weeks|{}'.format(np.round(x[ga_column_name],2), x.tag), axis=1)
# df = df.sort_values(by=ga_column_name, ascending=False)
# print(df[["ga_boe_weeks", "image_title"]].head())
# print(df.shape)
# print(df.columns)
# print(df.head)
condition1 = df[originalCol] == 1
condition2 = df[originalCol] == 0
df1 = df[condition1]
df2 = df[condition2]

data_rootpath = "/mnt/raid/C1_ML_Analysis/"
display_sample_images(image_frames_info=df1, 
                                images_rootpath=data_rootpath, 
                                img_path_column=img_path_column,
                                highlight_column = compareCol,
                                savefig_filepath=None,
                                ga_col = ga_column_name,
                                plt_title = originalCol + " Ground Truth = 1",
                                savefig_filepath = "/mnt/raid/home/ayrisbud/us-famli-pl/src/classification_samples/" + originalCol + "_Ground Truth_1.png"
                                )

display_sample_images(image_frames_info=df2, 
                                images_rootpath=data_rootpath, 
                                img_path_column=img_path_column,
                                highlight_column = compareCol,
                                savefig_filepath=None,
                                ga_col = ga_column_name,
                                plt_title = originalCol + " Ground Truth = 0",
                                savefig_filepath = "/mnt/raid/home/ayrisbud/us-famli-pl/src/classification_samples/" + originalCol + "_Ground Truth_0.png"
                                )

In [ ]:
img_path_column = 'img_path'
ga_column_name = 'ga_boe_weeks'
originalCol = classif_error_column[3]
predictedCol = classif_error_column[3] + "_predicted"
compareCol = classif_error_column[3] + "_compare"
# print("Entered i: ", originalCol, " ", predictedCol, " ", compareCol)
df[ga_column_name] = (df.ga_boe / 7.0).fillna(value=-1)
# df['is_measurable_head'] = df['Head Measurable'] == "YES"
df[compareCol] = [True if original == predicted else False for original, predicted in zip(df[originalCol], df[predictedCol])]
df['image_title'] = df.apply(lambda x: '{}weeks|{}'.format(np.round(x[ga_column_name],2), x.tag), axis=1)
# df = df.sort_values(by=ga_column_name, ascending=False)
# print(df[["ga_boe_weeks", "image_title"]].head())
# print(df.shape)
# print(df.columns)
# print(df.head)
condition1 = df[originalCol] == 1
condition2 = df[originalCol] == 0
df1 = df[condition1]
df2 = df[condition2]

data_rootpath = "/mnt/raid/C1_ML_Analysis/"
display_sample_images(image_frames_info=df1, 
                                images_rootpath=data_rootpath, 
                                img_path_column=img_path_column,
                                highlight_column = compareCol,
                                savefig_filepath=None,
                                ga_col = ga_column_name,
                                plt_title = originalCol + " Ground Truth = 1",
                                savefig_filepath = "/mnt/raid/home/ayrisbud/us-famli-pl/src/classification_samples/" + originalCol + "_Ground Truth_1.png"
                                )

display_sample_images(image_frames_info=df2, 
                                images_rootpath=data_rootpath, 
                                img_path_column=img_path_column,
                                highlight_column = compareCol,
                                savefig_filepath=None,
                                ga_col = ga_column_name,
                                plt_title = originalCol + " Ground Truth = 0",
                                savefig_filepath = "/mnt/raid/home/ayrisbud/us-famli-pl/src/classification_samples/" + originalCol + "_Ground Truth_0.png"
                                )

In [ ]:
img_path_column = 'img_path'
ga_column_name = 'ga_boe_weeks'
originalCol = classif_error_column[4]
predictedCol = classif_error_column[4] + "_predicted"
compareCol = classif_error_column[4] + "_compare"
# print("Entered i: ", originalCol, " ", predictedCol, " ", compareCol)
df[ga_column_name] = (df.ga_boe / 7.0).fillna(value=-1)
# df['is_measurable_head'] = df['Head Measurable'] == "YES"
df[compareCol] = [True if original == predicted else False for original, predicted in zip(df[originalCol], df[predictedCol])]
df['image_title'] = df.apply(lambda x: '{}weeks|{}'.format(np.round(x[ga_column_name],2), x.tag), axis=1)
# df = df.sort_values(by=ga_column_name, ascending=False)
# print(df[["ga_boe_weeks", "image_title"]].head())
# print(df.shape)
# print(df.columns)
# print(df.head)
condition1 = df[originalCol] == 1
condition2 = df[originalCol] == 0
df1 = df[condition1]
df2 = df[condition2]

data_rootpath = "/mnt/raid/C1_ML_Analysis/"
display_sample_images(image_frames_info=df1, 
                                images_rootpath=data_rootpath, 
                                img_path_column=img_path_column,
                                highlight_column = compareCol,
                                savefig_filepath=None,
                                ga_col = ga_column_name,
                                plt_title = originalCol + " Ground Truth = 1",
                                savefig_filepath = "/mnt/raid/home/ayrisbud/us-famli-pl/src/classification_samples/" + originalCol + "_Ground Truth_1.png"
                                )

display_sample_images(image_frames_info=df2, 
                                images_rootpath=data_rootpath, 
                                img_path_column=img_path_column,
                                highlight_column = compareCol,
                                savefig_filepath=None,
                                ga_col = ga_column_name,
                                plt_title = originalCol + " Ground Truth = 0",
                                savefig_filepath = "/mnt/raid/home/ayrisbud/us-famli-pl/src/classification_samples/" + originalCol + "_Ground Truth_0.png"
                                )

In [ ]:
img_path_column = 'img_path'
ga_column_name = 'ga_boe_weeks'
originalCol = classif_error_column[5]
predictedCol = classif_error_column[5] + "_predicted"
compareCol = classif_error_column[5] + "_compare"
# print("Entered i: ", originalCol, " ", predictedCol, " ", compareCol)
df[ga_column_name] = (df.ga_boe / 7.0).fillna(value=-1)
# df['is_measurable_head'] = df['Head Measurable'] == "YES"
df[compareCol] = [True if original == predicted else False for original, predicted in zip(df[originalCol], df[predictedCol])]
df['image_title'] = df.apply(lambda x: '{}weeks|{}'.format(np.round(x[ga_column_name],2), x.tag), axis=1)
# df = df.sort_values(by=ga_column_name, ascending=False)
# print(df[["ga_boe_weeks", "image_title"]].head())
# print(df.shape)
# print(df.columns)
# print(df.head)
condition1 = df[originalCol] == 1
condition2 = df[originalCol] == 0
df1 = df[condition1]
df2 = df[condition2]

data_rootpath = "/mnt/raid/C1_ML_Analysis/"
display_sample_images(image_frames_info=df1, 
                                images_rootpath=data_rootpath, 
                                img_path_column=img_path_column,
                                highlight_column = compareCol,
                                savefig_filepath=None,
                                ga_col = ga_column_name,
                                plt_title = originalCol + " Ground Truth = 1",
                                savefig_filepath = "/mnt/raid/home/ayrisbud/us-famli-pl/src/classification_samples/" + originalCol + "_Ground Truth_1.png"
                                )

display_sample_images(image_frames_info=df2, 
                                images_rootpath=data_rootpath, 
                                img_path_column=img_path_column,
                                highlight_column = compareCol,
                                savefig_filepath=None,
                                ga_col = ga_column_name,
                                plt_title = originalCol + " Ground Truth = 0",
                                savefig_filepath = "/mnt/raid/home/ayrisbud/us-famli-pl/src/classification_samples/" + originalCol + "_Ground Truth_0.png"
                                )

In [ ]:
import numpy as np
from sklearn.datasets import make_multilabel_classification
from sklearn.model_selection import train_test_split

# Create a synthetic multilabel dataset
X, y = make_multilabel_classification(n_samples=1000, n_features=10, n_classes=5, n_labels=2, random_state=42)
print(y)

In [ ]:
Xtrain, Xtest, Yttain, ytest = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)